In [7]:
import numpy as np
from ecog_band.utils import *
import pandas as pd
import numpy as np
import scipy.io as scio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import cm
from matplotlib.cm import ScalarMappable
from ecog_band.datasetAllband import SVMDataset
from ecog_band.models import SVMBinClassifier
import os
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from ecog_band.utils import *
from ecog_band.solver import Nfold_solver
import pandas as pd
# from ecog_band.datasetExcludeBand import CustomDatasetExcband
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix
import matplotlib.image as mpimg
from matplotlib.patches import Wedge

HS = 69
freq = 500

rootPath = '/root/pp/covert-reading/Ecog_pretrain/accuracy_results_svm_avgfreq'
path = "/public/DATA/overt_reading/"

band_list = get_all_band()
n_freq_bands = len(band_list)
idx_elec = list(range(1, 256))

In [3]:
# 计算all_bands acc
for elec in idx_elec:
    path_elec = f'/public/DATA/overt_reading/dataset_/HS{HS}/{freq}/{elec}'
    num_samples = len(os.listdir(path_elec))
    y_save_path = f'/root/pp/covert-reading/Ecog_pretrain/accuracy_results_svm_avgfreq/HS{HS}/{freq}/{elec}'
    os.makedirs(y_save_path, exist_ok=True)

    data_loader = SVMDataset(HS, path_elec, freq, elec, num_samples, avg='avgFreq')
    # data_loader = CombineBandDataset(HS, path_elec, freq, elec, num_samples, band_list, avg='avgFreq')
    data, labels = data_loader.get_data_labels()
    # print(f'data_shape: {data.shape}')

    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=1/6, random_state=42)
    # print(f'x_train shape：{X_train.shape}, x_test shape：{X_test.shape}, y_train shape：{y_train.shape}, y_test shape：{y_test.shape}')# (600, 375), x_test shape：(120, 375), y_train shape：(600,), y_test shape：(120,)

    svm = SVMBinClassifier()
    # svm = DecisionTreeBinClassifier()
    svm.train(x_train=X_train, y_train=y_train)

    y_pred = svm.evaluate(X_test=X_test, y_test=y_test)
    all_band_acc = accuracy_score(y_test, y_pred)

    np.save(os.path.join(y_save_path, f'allbands_y_pred.npy'), y_pred)
    np.save(os.path.join(y_save_path, f'allbands_y_true.npy'), y_test)

    print(f"Accuracy on test set - elec{elec}: {all_band_acc}")


Accuracy on test set - elec1: 0.8083333333333333
Accuracy on test set - elec2: 0.7916666666666666
Accuracy on test set - elec3: 0.9666666666666667
Accuracy on test set - elec4: 0.6083333333333333
Accuracy on test set - elec5: 0.675
Accuracy on test set - elec6: 0.48333333333333334
Accuracy on test set - elec7: 0.7333333333333333
Accuracy on test set - elec8: 0.925
Accuracy on test set - elec9: 0.9166666666666666
Accuracy on test set - elec10: 0.8
Accuracy on test set - elec11: 0.8416666666666667
Accuracy on test set - elec12: 0.65
Accuracy on test set - elec13: 0.6583333333333333
Accuracy on test set - elec14: 0.625
Accuracy on test set - elec15: 0.6916666666666667
Accuracy on test set - elec16: 0.49166666666666664
Accuracy on test set - elec17: 0.6583333333333333
Accuracy on test set - elec18: 0.575
Accuracy on test set - elec19: 0.8
Accuracy on test set - elec20: 0.65
Accuracy on test set - elec21: 0.5083333333333333
Accuracy on test set - elec22: 0.4666666666666667
Accuracy on test 

In [ ]:

# 所有电极在不同频段的contribution
for elec in idx_elec:
    path_elec = f'/public/DATA/overt_reading/dataset_/HS{HS}/{freq}/{elec}'
    num_samples = len(os.listdir(path_elec))
    y_save_path = f'/root/pp/covert-reading/Ecog_pretrain/accuracy_results_svm_avgfreq/HS{HS}/{freq}/{elec}'
    fig_save_path = f'/root/pp/covert-reading/Ecog_pretrain/accuracy_results_svm_avgfreq/HS{HS}/{freq}/{elec}/figs'
    contribution_save_path = f'/root/pp/covert-reading/Ecog_pretrain/contribution_results_svm_avgfreq/HS{HS}/{freq}/{elec}'
    os.makedirs(y_save_path, exist_ok=True)
    os.makedirs(fig_save_path, exist_ok=True)
    os.makedirs(contribution_save_path, exist_ok=True)
    baseline_accuracy = cal_acc_band1_from_y(y_save_path, 'allbands')
    contributions_list = []
    for band in band_list:
        data_loader = SVMDataset(HS, path_elec, freq, elec, num_samples, band, exclude=True, avg='avgFreq')
        data, labels = data_loader.get_data_labels()
        print(f'data_shape: {data.shape}')
        X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=1/6, random_state=42)
        # svm = DecisionTreeBinClassifier()
        svm = SVMBinClassifier()
        svm.train(x_train=X_train, y_train=y_train)

        y_pred = svm.evaluate(X_test=X_test, y_test=y_test)
        band_acc = accuracy_score(y_test, y_pred)
        print(f"Accuracy on test set without band_{band}: {band_acc}")

        np.save(os.path.join(y_save_path, f'{band}_y_pred.npy'), y_pred)
        np.save(os.path.join(y_save_path, f'{band}_y_true.npy'), y_test)
        # plt confusion matrix
        plt_confusion_matric(y_test, y_pred, HS, elec, freq, f'without_{band}')

        band_acc = cal_acc_band1_from_y(y_save_path, band)
        contribution = baseline_accuracy - band_acc
        print(f'band: {band}, contribution_acc:{contribution}')
        contributions_list.append(contribution)
    np.save(os.path.join(contribution_save_path, f'contributions.npy'), contributions_list)

In [8]:
contribution_dic = {}
for elec in idx_elec:
    contribution_save_path = f'/root/pp/covert-reading/Ecog_pretrain/contribution_results_svm_avgfreq/HS{HS}/{freq}/{elec}'
    contribution_list = np.load(os.path.join(contribution_save_path, f'contributions.npy'))
    contribution_dic[elec] = contribution_list

In [17]:

# 数据加载
subject_str = f"HS{HS}"
xy = scio.loadmat(path + f"/elecs/All_elecs/{subject_str}_warped_elec_pos2D.mat")['elecmatrix']
anatomy = scio.loadmat(path + f"/elecs/All_elecs/{subject_str}_elecs_all_warped.mat")['anatomy']
# sig_elecs_HS = np.load(path + f"/elecs/sig_elecs/{subject_str}_sig_elecs5.npy", allow_pickle=True).item()

# 颜色映射
band_colors = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#FF00FF', '#00FFFF', '#FF8000', '#8000FF']

# band_colors = ['mako', 'Accent', 'Blues', 'Dark2', 'Greens', 'Oranges', 'RdBu', 'Reds']

markers = ['o', 's', 'D', '^', 'v', 'P', '*', 'X']  # 不同的标记形状
# 加载大脑模板图像
img = mpimg.imread(path + "/elecs/Brain2D/MNI152_brain2D.png")

# fig, ax = plt.subplots(figsize=(15, 15))
# ax.imshow(img)
fig, ax = plt.subplots(figsize=(12, 10))
ax.imshow(img)

# 绘制每个频段的电极
# 绘制每个电极的扇形图
for elec in idx_elec:
    x, y = xy[elec]
    
    # 获取该电极的分类贡献度
    contributions = contribution_dic[elec]

    # 处理负值，将所有贡献度都变为正数
    min_contribution = min(contributions, default=0)
    if min_contribution < 0:
        for bandid in range(len(contributions)):
            contributions[bandid] -= min_contribution
    total_contribution = sum(contributions)
    
    if total_contribution > 0:
        # 计算扇形图的角度
        angles = [contributions[band] / total_contribution * 360 for band in range(n_freq_bands)]
        colors = band_colors  # 每个频段的颜色
        
        start_angle = 0
        for i, angle in enumerate(angles):
            wedge = Wedge(
                center=(x, y),
                r=5,  # 扇形图半径，可以根据需要调整
                theta1=start_angle,
                theta2=start_angle + angle,
                color=colors[i],
                alpha=0.7,
                edgecolor='k'
            )
            ax.add_patch(wedge)
            start_angle += angle
    # sc = ax.scatter(x_coords, y_coords, c=accuracy_values, cmap=color_maps[id], norm=norms[id], alpha=0.5, s=100, label=f'Band {band}')

handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=band_colors[i], markersize=10, label=band_list[i]) for i in range(len(band_list))]
ax.legend(handles=handles, title="Frequency Bands", bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
ax.set_title(f"Electrodes Distribution with Frequency Band Contribution for HS{HS}", size=22)

plt.tight_layout(rect=[0, 0, 1, 0.9])
plt.savefig(f"/root/pp/covert-reading/Ecog_pretrain/figures/fig_contri_distri_HS{HS}_freq{freq}.pdf", dpi=1200)
plt.show()

/tmp/ipykernel_21233/2254770050.py:43: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  wedge = Wedge(
